# Question 3: 線形計画法 (3)

=============================================================================================

## 食事問題
https://www.y-shinno.com/pulp-intro/

問題）食品A,B,Cの３種類を組み合わせて摂取する。  
栄養素摂取量を満たした上で，食費を最小限にできる摂取量を求める。  

●食品  
・食品A：価格=20, 栄養素1=22, 栄養素2=20, 栄養素3=10  
・食品B：価格=12, 栄養素1=13, 栄養素2=30, 栄養素3=5  
・食品C：価格=18, 栄養素1=17, 栄養素2=5, 栄養素3=12  

●必要な栄養素摂取量  
・栄養素１：200  
・栄養素２：200  
・栄養素３：100  

数理最適化のライブラリー pulp とCVXPY を用いて解きなさい。

=============================================================================================

定式化）  
この場合，目的関数は食品Aの重量をXa(g)，BをXb(g)，CをXc(g)とした場合，以下のようになります：

【目的関数】  
f = 20xa + 12xb + 18xc (価格）  

また満たさなければならない制約としては以下のようになります：

【制約条件】  
22xa + 13xb + 17xc >= 200（栄養素１）  
20xa + 30xb + 5xc  >= 200（栄養素２）  
10xa + 3xb  + 12xc >= 100（栄養素３）

つまり，制約条件をすべて満たした上で，価格が最小になる食品の重量を算出すれば良いわけですね。

定式化）  
この場合，目的関数は食品Aの重量をXa(g)，BをXb(g)，CをXc(g)とした場合，以下のようになります：

【目的関数】  
f = 20xa + 12xb + 18xc (価格）  

また満たさなければならない制約としては以下のようになります：

【制約条件】  
22xa + 13xb + 17xc >= 200（栄養素１）  
20xa + 30xb + 5xc  >= 200（栄養素２）  
10xa + 3xb  + 12xc >= 100（栄養素３）

つまり，制約条件をすべて満たした上で，価格が最小になる食品の重量を算出すれば良いわけですね。from pulp import *  # conda install -c conda-forge pulp

In [1]:
!pip install pulp
from pulp import * 

In [2]:
# 問題の定義
problem = LpProblem(name="Diet", sense=LpMinimize)

# 変数の定義
A = LpVariable(name = "A", lowBound = 0, cat="Integer") # lowBoundで０以上を定義
B = LpVariable(name = "B", lowBound = 0, cat="Integer")
C = LpVariable(name = "C", lowBound = 0, cat="Integer")

# 目的関数の定義
problem += 20*A + 12*B + 18*C

# 制約条件の定義
problem += 22*A + 13*B + 17*C >= 200
problem += 20*A + 30*B + 5*C >= 200
problem += 10*A + 5*B + 12*C >= 100

print(problem)

Diet:
MINIMIZE
20*A + 12*B + 18*C + 0
SUBJECT TO
_C1: 22 A + 13 B + 17 C >= 200

_C2: 20 A + 30 B + 5 C >= 200

_C3: 10 A + 5 B + 12 C >= 100

VARIABLES
0 <= A Integer
0 <= B Integer
0 <= C Integer



In [3]:
# 解く
status = problem.solve()
print(LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Oct 15 2020 

command line - cbc /tmp/364339e0ffe94387bda6f2db75f75130-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/364339e0ffe94387bda6f2db75f75130-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 27 RHS
At line 31 BOUNDS
At line 35 ENDATA
Problem MODEL has 3 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 189.067 - 0.00 seconds
Cgl0003I 0 fixed, 3 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 3 rows, 3 columns (3 integer (0 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 1.9999
Cbc0012I Integer solution of 200 found by greedy cover after 0 iterations and 0 nodes (0.02 seconds)
Cbc0012I Integer solution of 190 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search

解けた場合は「Optimal」と出力されます。

In [4]:
# 結果表示
print("Result")
print("A:", A.value())
print("B:", B.value())
print("C:", C.value())

Result
A: 5.0
B: 3.0
C: 3.0


次は CVXPY を用いて解いてみましょう。

In [5]:
import cvxpy as cp

In [6]:
# 変数の定義
A = cp.Variable(integer=True) # 整数として定義
B = cp.Variable(integer=True)
C = cp.Variable(integer=True)

# 制約条件の定義
constraints = [22*A + 13*B + 17*C >= 200,
               20*A + 30*B + 5*C >= 200,
               10*A + 5*B + 12*C >= 100,
               A >= 0,
               B >= 0,
               C >= 0]

# 目的関数の定義
obj = cp.Minimize(20*A + 12*B + 18*C)

# 問題の定義
prob = cp.Problem(obj, constraints)
print(prob)

minimize 20.0 @ var0 + 12.0 @ var1 + 18.0 @ var2
subject to 200.0 <= 22.0 @ var0 + 13.0 @ var1 + 17.0 @ var2
           200.0 <= 20.0 @ var0 + 30.0 @ var1 + 5.0 @ var2
           100.0 <= 10.0 @ var0 + 5.0 @ var1 + 12.0 @ var2
           0.0 <= var0
           0.0 <= var1
           0.0 <= var2


In [7]:
# 解く
prob.solve()  # Returns the optimal value.
print("status:", prob.status)

Long-step dual simplex will be used
status: optimal


In [8]:
# 結果表示
print("optimal value", prob.value)
print("A:", A.value)
print("B:", B.value)
print("C:", C.value)

optimal value 190.0
A: 5.0
B: 3.0
C: 3.0
